In [1]:
import pandas as pd
import numpy as np
import polars as pl

from m4_feats_polars import *
from m5_sb_models import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
data_path     = 'kaggle/input/linking-writing-processes-to-writing-quality/'
train_logs    = pl.scan_csv(f'{data_path}/train_logs.csv')
test_logs    = pl.scan_csv(f'{data_path}/test_logs.csv')
train_scores = pl.scan_csv(f'{data_path}/train_scores.csv')

In [3]:
# everything is logged - DONE
# bursts = 2/3 of a second - input only - DONE
# inter word pauses
# between sentence pauses ?
# between paragraph pauses ?
# backspace pauses
# edit pauses

In [4]:
# # TEST INDIVIDUAL FEATURES
# from m3_model_params import lgb_params_1
# tr_word_pause, ts_word_pause = word_pauses(train_logs, test_logs)
# train_feats = tr_word_pause.join(train_scores, on='id', how='left')
# test_feats = ts_word_pause.clone()

# train_feats = train_feats.collect().to_pandas()
# test_feats = test_feats.collect().to_pandas()
# test_preds, valid_preds, final_rmse, cv_rm = lgb_pipeline(train_feats, test_feats, lgb_params_1)

In [5]:
# [('train_down_events_counts.pkl', 26),
#  ('train_vector_one_gram.pkl', 26),
#  ('train_create_pauses.pkl', 26),
#  ('train_essay_paragraphs.pkl', 26),
#  ('train_cursor_pos_acceleration.pkl', 11),
#  ('train_word_count_acceleration.pkl', 6),
#  ('train_p_burst_feats.pkl', 5),
#  ('train_r_burst_feats.pkl', 3),
#  ('train_events_counts_acceleration.pkl', 3),
#  ('train_essay_sentences.pkl', 3),
#  ('train_categorical_nunique.pkl', 3),
#  ('train_vector_two_gram.pkl', 2),
#  ('train_cursor_pos_rate_of_change.pkl', 2),
#  ('train_word_counts_rate_of_change.pkl', 2),
#  ('train_count_of_activities.pkl', 2),
#  ('train_action_time_by_activity.pkl', 1),
#  ('train_product_to_keys.pkl', 1),
#  ('train_IKI_based_fractals.pkl', 1),
#  ('train_events_counts_baseline.pkl', 1)]

In [6]:
# best_feature_set_1 - PARTIAL
train_essays          = get_essay_df(train_logs.collect().to_pandas())
test_essays           = get_essay_df(test_logs.collect().to_pandas())

tr_down_events_counts, ts_down_events_counts = down_events_counts(train_logs, test_logs)
tr_vect_one, ts_vect_one = countvectorize_one_one(train_essays, test_essays)
tr_pauses, ts_pauses = create_pauses(train_logs, test_logs)
tr_cursor_pos_acc, ts_cursor_pos_acc = cursor_pos_acceleration(train_logs, test_logs)
tr_word_pause, ts_word_pause = word_pauses(train_logs, test_logs)
# tr_word_count_acc, ts_word_count_acc = word_count_acceleration(train_logs, test_logs)
# tr_p_burst, ts_p_burst = p_burst_feats(train_logs, test_logs, 2)
# tr_r_burst, ts_r_burst = r_burst_feats(train_logs, test_logs)
#tr_event_acc, ts_event_acc = events_counts_acceleration(train_logs, test_logs)
# tr_nunique, ts_nunique = categorical_nunique(train_logs, test_logs)
# tr_vect_two, ts_vect_two = countvectorize_two_one(train_essays, test_essays)
# tr_time_by_act, ts_time_by_act = action_time_by_activity(train_logs, test_logs)
# tr_cursor_pos_roc, ts_cursor_pos_roc = cursor_pos_rate_of_change(train_logs, test_logs)
# 
# tr_act_count, ts_act_count = count_of_activities(train_logs, test_logs)
# tr_get_keys, ts_get_keys = get_keys_pressed_per_second(train_logs.collect().to_pandas(), 
#                                                        test_logs.collect().to_pandas())
# 
# tr_input_change, ts_input_change = input_text_change_feats(train_logs, test_logs)
# tr_wc_roc, ts_wc_roc =  word_counts_rate_of_change(train_logs, test_logs)


train_feats = tr_down_events_counts.join(tr_vect_one, on='id', how='left')
train_feats = train_feats.join(tr_pauses, on='id', how='left')
train_feats = train_feats.join(tr_cursor_pos_acc, on='id', how='left')
train_feats = train_feats.join(tr_word_pause, on='id', how='left')
# train_feats = train_feats.join(tr_word_count_acc, on='id', how='left')
# train_feats = train_feats.join(tr_vect_two, on='id', how='left')
# train_feats = train_feats.join(tr_p_burst, on='id', how='left')
# train_feats = train_feats.join(tr_r_burst, on='id', how='left')
# train_feats = train_feats.join(tr_event_acc, on='id', how='left')
# train_feats = train_feats.join(tr_nunique, on='id', how='left')
# train_feats = train_feats.join(tr_wc_roc, on='id', how='left')
# train_feats = train_feats.join(tr_act_count, on='id', how='left')
# train_feats = train_feats.join(tr_cursor_pos_roc, on='id', how='left')

# train_feats = train_feats.join(tr_get_keys, on='id', how='left')
# train_feats = train_feats.join(tr_input_change, on='id', how='left')
# train_feats = train_feats.join(tr_time_by_act, on='id', how='left')

test_feats = ts_down_events_counts.join(ts_vect_one, on='id', how='left')
test_feats = test_feats.join(ts_pauses, on='id', how='left')
test_feats = test_feats.join(ts_cursor_pos_acc, on='id', how='left')
test_feats = test_feats.join(ts_word_pause, on='id', how='left')
# test_feats = test_feats.join(ts_word_count_acc, on='id', how='left')
# test_feats = test_feats.join(ts_vect_two, on='id', how='left')
# test_feats = test_feats.join(ts_p_burst, on='id', how='left')
# test_feats = test_feats.join(ts_r_burst, on='id', how='left')
# test_feats = test_feats.join(tr_event_acc, on='id', how='left')
# test_feats = test_feats.join(ts_nunique, on='id', how='left')
# test_feats = test_feats.join(ts_wc_roc, on='id', how='left')
# test_feats = test_feats.join(ts_act_count, on='id', how='left')
# test_feats = test_feats.join(ts_cursor_pos_roc, on='id', how='left')


# test_feats = test_feats.join(ts_get_keys, on='id', how='left')
# test_feats = test_feats.join(ts_input_change, on='id', how='left')
# test_feats = test_feats.join(ts_time_by_act, on='id', how='left')


train_logs = train_logs.collect().to_pandas()
test_logs = test_logs.collect().to_pandas()
train_scores = train_scores.collect().to_pandas()
train_feats = train_feats.sort('id')
train_feats = train_feats.collect().to_pandas()
test_feats = test_feats.collect().to_pandas()

train_feats           = train_feats.merge(parag_feats(train_essays), on='id', how='left')
test_feats            = test_feats.merge(parag_feats(test_essays), on='id', how='left')

# train_feats           = train_feats.merge(sent_feats(train_essays), on='id', how='left')
# test_feats            = test_feats.merge(sent_feats(test_essays), on='id', how='left')

train_feats = train_feats.merge(train_scores, on='id', how='left')
print(f'train feats shape {train_feats.shape}')


< Events counts features >
< Count vectorize one-grams >
< Idle time features >
< cursor position acceleration >
word pauses
< Essays paragraphs feats >
< Essays paragraphs feats >
train feats shape (2471, 79)


In [7]:
from m5_sb_models import lgb_pipeline
lgb_params_1 = {
    'boosting_type': 'gbdt', 
    'metric': 'rmse',
    'reg_alpha': 0.0031, 
    'reg_lambda': 0.001, 
    'colsample_bytree': 0.8,  
    'subsample_freq': 1,  
    'subsample': 0.75,  
    'learning_rate': 0.017, 
    'num_leaves': 19, 
    'min_child_samples': 46,
    'n_estimators': 350,
    'verbosity': -1
    }

param = {'n_estimators': 1024,
        'learning_rate': 0.005,
        'metric': 'rmse',
        'force_col_wise': True,
        'verbosity': 0,}

# train_feats = train_feats[['id', 'score'] + feat_select]
# test_feats = test_feats[['id'] + feat_select]

print(f'train feats shape {train_feats.shape}')


train feats shape (2471, 79)


In [8]:
shuffle_preds = []

for i in range(15):
    train_feats = train_feats.sample(frac=1).reset_index(drop=True)
    test_preds, oof_preds, rmse, model = lgb_pipeline(train_feats, test_feats, lgb_params_1)
    shuffle_preds.append(rmse)
    #test_preds, oof_preds, rmse, model = lgb_pipeline(train_feats, test_feats, param)

np.mean(shuffle_preds)

Final RMSE over 50: 0.605581. Std 0.8253
RMSE by fold 0.605397. Std 0.0146
Final RMSE over 50: 0.605680. Std 0.8246
RMSE by fold 0.605628. Std 0.0079
Final RMSE over 50: 0.603056. Std 0.8242
RMSE by fold 0.602989. Std 0.0090
Final RMSE over 50: 0.604434. Std 0.8241
RMSE by fold 0.604284. Std 0.0133
Final RMSE over 50: 0.604323. Std 0.8246
RMSE by fold 0.604130. Std 0.0151
Final RMSE over 50: 0.604961. Std 0.8237
RMSE by fold 0.604798. Std 0.0139
Final RMSE over 50: 0.603751. Std 0.8249
RMSE by fold 0.603644. Std 0.0109
Final RMSE over 50: 0.604211. Std 0.8252
RMSE by fold 0.604013. Std 0.0153
Final RMSE over 50: 0.604329. Std 0.8240
RMSE by fold 0.604178. Std 0.0134
Final RMSE over 50: 0.604994. Std 0.8233
RMSE by fold 0.604875. Std 0.0119
Final RMSE over 50: 0.604312. Std 0.8242
RMSE by fold 0.604145. Std 0.0139
Final RMSE over 50: 0.604664. Std 0.8254
RMSE by fold 0.604584. Std 0.0101
Final RMSE over 50: 0.604700. Std 0.8245
RMSE by fold 0.604618. Std 0.0102
Final RMSE over 50: 0.606

0.6046250116470197

In [ ]:
0.6046250116470197

- M4 + cursor_pos_acc + word_count_acc + bigrams <br />
0.6044949502480581
- M4 + cursor_pos_acc + word_count_acc + nunique <br />
0.6048063065033383
- M4 + cursor_pos_acc + word_count_acc + sentences <br />
0.6050618131109056
- M4 + cursor_pos_acc + word_count_acc + event_id_acceleration <br />
0.6052
- M4 + cursor_pos_acc + word_count_acc + r_burst <br />
0.6050068640389566
- M4 + cursor_pos_acc + word_count_acc + p_burst <br />
0.605038295599663
- M4 + cursor_pos_acc + word_count_acc  <br />
0.6047963953877546 <br />
- M4 + cursor_pos_acc <br />
0.6054500859298344 <br />
- Magic 4 <br />
0.6062498923247683 <br />

In [9]:
oof_res = oof_preds.groupby(['id', 'score'])['preds'].mean().reset_index()
# oof_res['rmse'] = oof_res.apply(lambda x: np.sqrt((x['score']-x['preds'])**2))
oof_res['RMSE'] = np.sqrt((oof_res['score']-oof_res['preds'])**2)
oof_res.groupby(['score'])['RMSE'].mean().reset_index().sort_values('RMSE', ascending=False)

,score,RMSE
0,0.5,1.496408
1,1.0,1.285115
11,6.0,1.129482
2,1.5,0.907662
10,5.5,0.747165
3,2.0,0.588413
9,5.0,0.460736
4,2.5,0.447573
5,3.0,0.444592
6,3.5,0.395458


In [10]:
oof_res = oof_preds.groupby(['id', 'score'])['preds'].mean().reset_index()
oof_res['RMSE'] = np.sqrt((oof_res['score']-oof_res['preds'])**2)
oof_res.groupby(['score'])['RMSE'].mean().reset_index().sort_values('RMSE', ascending=False)

,score,RMSE
0,0.5,1.496408
1,1.0,1.285115
11,6.0,1.129482
2,1.5,0.907662
10,5.5,0.747165
3,2.0,0.588413
9,5.0,0.460736
4,2.5,0.447573
5,3.0,0.444592
6,3.5,0.395458


In [11]:
oof_res = oof_preds.groupby(['id', 'score'])['preds'].mean().reset_index()
# oof_res['rmse'] = oof_res.apply(lambda x: np.sqrt((x['score']-x['preds'])**2))
oof_res['RMSE'] = np.sqrt((oof_res['score']-oof_res['preds'])**2)

In [12]:
import lightgbm as lgb

weights = [1.0 for _ in range(len(y))]  # Default weights
weights[some_specific_index] = 1.5  # Higher weight for a specific instance

train_data = lgb.Dataset(X_train, label=y_train, weight=weights)
# Proceed with setting up parameters and training the model


NameError: name 'y' is not defined

train feats shape (2471, 151)
Final RMSE over 50: 0.605768. Std 0.8244
RMSE by fold 0.605635. Std 0.0128

reduced feats
Final RMSE over 50: 0.604472. Std 0.8245
RMSE by fold 0.604365. Std 0.0115

r burst only
Number of estimators: 350
Final RMSE over 50: 0.604057. Std 0.8244
RMSE by fold 0.603946. Std 0.0116

Number of estimators: 350
Final RMSE over 50: 0.604650. Std 0.8243
RMSE by fold 0.604527. Std 0.0122

In [ ]:
test_ids = test_feats.id
y_pred = np.mean(test_preds, axis=0)

sub = pd.DataFrame({'id': test_ids, 'score': y_pred})
sub.to_csv('submission.csv', index=False)